In [ ]:
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import os

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv('OPENAI_API_KEY'),
)

qdrant_client= QdrantClient(url = 'http://localhost:6333')


In [ ]:
def get_embedding(input:str):
    embedding = client.embeddings.create(
        model="baai/bge-base-en-v1.5",
        input= input,
        encoding_format="float"
        )
    return embedding.data[0].embedding


In [ ]:
get_embedding('what kind of earphone do i get ?')

In [ ]:
def retrieve_data(input:str, qdrant_client=qdrant_client):
    query_embedding = get_embedding(input)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection",
        query=query_embedding,
        with_payload=True,
        limit=5
    ).points

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
retrieved_context = retrieve_data('what kind of ear phones do i get?', qdrant_client)

In [ ]:
retrieved_context

Format retrieved context function

In [ ]:
def process_context(context):
    formatted_context = ""
    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"],context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"
    return formatted_context

In [ ]:
processed_context = process_context(retrieved_context)

In [ ]:
print(processed_context)

In [ ]:
def build_prompt(question, context):
    return f"""You are ai shopping assistant that can answer question about the products in stock
You will be given a question and context.
Instructions:
1. You need to asnwer based on the context only
2. never use the word context and refer it as available product

Context:
{context}

Question:
{question}"""

In [ ]:
prompt = build_prompt("what kind of ear phones do i get?", processed_context)
print(prompt)

Generate answer

In [ ]:
def generate_answer(prompt):
    response = client.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[{"role": "system", "content": prompt}],
        extra_body={"reasoning": {"enabled": False}}
        )
    response = response.choices[0].message
    return response

In [ ]:
print(generate_answer(prompt))

Combined Pipeline

In [ ]:
def rag_pipeline(question):
    """RAG pipeline."""
    retrieved_context = retrieve_data(question, qdrant_client)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(question, preprocessed_context)
    answer = generate_answer(prompt)

    return answer


In [ ]:
print(rag_pipeline('what kind of earphones do i get?'))